In [1]:
import time

import redis

import cst_python as cst
from cst_python.memory_storage import MemoryStorageCodelet

In [2]:
client = redis.Redis(decode_responses=True)
client.flushall()

True

```mermaid
flowchart LR

update[Update Memory]
send[Send Memory]
retrieve[Retrieve Memory]
request[Request Memory]
handler_notify_transfer[Handler: Notify Transfer]
handler_transfer_memory[Handler: Transfer Memory]


update --> |"timestamp(MS) < timestamp(local)"| send
update --> |"timestamp(MS) > timestamp(local)"| retrieve





handler_transfer_memory --> send

subgraph retrieveContext
retrieve --> |owner is not empty| request

request -.->|Wait transfer event| handler_notify_transfer

end

```

In [2]:
mind = cst.Mind()
memory1 = mind.create_memory_object("Memory1", "")

In [3]:
ms_codelet = MemoryStorageCodelet(mind, "node0")
ms_codelet.time_step = 100

mind.insert_codelet(ms_codelet)
mind.start()

node03 Retrieve Memory1


In [4]:
memory1

MemoryObject [idmemoryobject=1, timestamp=1727456263799, evaluation=0.0, I=[1, 1, 1], name=Memory1]

In [6]:
memory1.set_info([1,1,1])

-1

node02 Updating memory Memory1
node02 Send memory Memory1
node02 Updating memory Memory1


In [7]:
time.sleep(1)

In [8]:
client.smembers("default_mind:nodes")

{'node0'}

In [9]:
client.hgetall("default_mind:memories:Memory1")

{'name': 'Memory1',
 'evaluation': '0.0',
 'I': '',
 'id': '0.0',
 'owner': 'node0'}

In [10]:
mind2 = cst.Mind()
mind2_memory1 = mind2.create_memory_object("Memory1", "")
mind2_ms_codelet = MemoryStorageCodelet(mind2)
mind2_ms_codelet.time_step = 100
mind2.insert_codelet(mind2_ms_codelet)
mind2.start()

In [11]:
mind2_memory1

node1 Retrieve Memory1
node1 Requesting Memory1
node0 Tranfering Memory1
node0 Send memory Memory1
node1 Updating memory Memory1
node0 Updating memory Memory1
node1 Send memory Memory1
node1 Updating memory Memory1
node0 Updating memory Memory1
node0 Retrieve Memory1
node0 INFO  ""
node1 INFO  ""


MemoryObject [idmemoryobject=0.0, timestamp=1726077369.7999365, evaluation=0.0, I=, name=Memory1]

In [12]:
mind2_ms_codelet._node_name

'node1'

In [13]:
client.smembers("default_mind:nodes")

{'node0', 'node1'}

In [14]:
time.sleep(1)

In [15]:
mind2_memory1

MemoryObject [idmemoryobject=0.0, timestamp=1726077369.7999365, evaluation=0.0, I=, name=Memory1]

In [16]:
client.hgetall("default_mind:memories:Memory1")

{'name': 'Memory1',
 'evaluation': '0.0',
 'I': '""',
 'id': '0',
 'owner': '',
 'timestamp': '1726077369.5866976'}

In [17]:
memory1.set_info("INFO")

-1

In [18]:
time.sleep(1)

node0 Updating memory Memory1
node0 Send memory Memory1
node1 Updating memory Memory1
node0 Updating memory Memory1
node1 Retrieve Memory1
node1 INFO INFO "INFO"


In [19]:
client.hgetall("default_mind:memories:Memory1")

{'name': 'Memory1',
 'evaluation': '0.0',
 'I': '"INFO"',
 'id': '0.0',
 'owner': '',
 'timestamp': '1726077370.926107'}

In [20]:
mind2_memory1

MemoryObject [idmemoryobject=0.0, timestamp=1726077371.003417, evaluation=0.0, I=INFO, name=Memory1]

In [21]:
time.sleep(1)

In [22]:
client.hgetall("default_mind:memories:Memory1")

{'name': 'Memory1',
 'evaluation': '0.0',
 'I': '"INFO"',
 'id': '0.0',
 'owner': '',
 'timestamp': '1726077370.926107'}

In [23]:
mind2_memory1.set_info("INFO2")
time.sleep(1)

node1 Updating memory Memory1
node1 Send memory Memory1
node0 Updating memory Memory1
node1 Updating memory Memory1
node0 Retrieve Memory1
node0 INFO INFO2 "INFO2"


In [24]:
client.hgetall("default_mind:memories:Memory1")

{'name': 'Memory1',
 'evaluation': '0.0',
 'I': '"INFO2"',
 'id': '0.0',
 'owner': '',
 'timestamp': '1726077373.0085642'}

In [25]:
memory1

MemoryObject [idmemoryobject=0.0, timestamp=1726077373.1104536, evaluation=0.0, I=INFO2, name=Memory1]

In [26]:
memory1.set_info(1)
time.sleep(1)

node0 Updating memory Memory1
node0 Send memory Memory1
node0 Updating memory Memory1
node1 Updating memory Memory1
node1 Retrieve Memory1
node1 INFO 1 1


In [27]:
mind2_memory1.get_info()

1

In [28]:
memory1.set_info("1")
time.sleep(1)

node0 Updating memory Memory1
node0 Send memory Memory1
node0 Updating memory Memory1
node1 Updating memory Memory1
node1 Retrieve Memory1
node1 INFO 1 "1"


In [29]:
mind2_memory1.get_info()

'1'

In [30]:
memory1.set_info(True)
time.sleep(1)


node0 Updating memory Memory1
node0 Send memory Memory1
node1 Updating memory Memory1
node0 Updating memory Memory1
node1 Retrieve Memory1
node1 INFO True true


In [31]:
mind2_memory1.get_info(), type(mind2_memory1.get_info())

(True, bool)

In [32]:
memory1.set_info([1,2,3])
time.sleep(1)
mind2_memory1.get_info(), type(mind2_memory1.get_info())

node0 Updating memory Memory1
node0 Send memory Memory1
node1 Updating memory Memory1
node0 Updating memory Memory1
node1 Retrieve Memory1
node1 INFO [1, 2, 3] [1, 2, 3]


([1, 2, 3], list)